In [ ]:
# 자동 미분을 사용하여 그레디언트 계산

In [1]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [2]:
(f(w1, w2 + eps) - f(w1, w2)) / eps

10.000000003174137

In [3]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

NameError: ignored